<a href="https://colab.research.google.com/github/chauhan01/Airlines-customer-segmentation-clustering/blob/master/instacart_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# setting up working directory

%cd /content/drive/My Drive/Data/Instacart

/content/drive/My Drive/Data/Instacart


In [0]:
#importing libraries

import pandas as pd
import numpy as np

In [0]:
# reding csv files

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head(2)

,user_id,product_id,product_bought_count,total_orders,average_duration_btw_orders,total_products_ordered,num_distinct_products,avg_products_in_basket,orders,reorders,reorder_rate,avg_pos_cart,aisle,department,eval_set,order_id,reordered
0,1,196,10,11,19.0,59,18,5.363636,35791,27791,0.776480,3.721774,soft drinks,beverages,train,1187899,1.0
1,1,10258,9,11,19.0,59,18,5.363636,1946,1389,0.713772,4.277492,nuts seeds dried fruit,snacks,train,1187899,1.0


In [6]:
print('train shape is ', train.shape)
print('test shape is ', test.shape)

train shape is  (8474661, 17)
test shape is  (4833292, 16)


In [7]:
# checking for missing values
print('missing values in train set\n',train.isna().sum())
print('\n\nmissing values in test set\n',test.isna().sum())

missing values in train set
 user_id                              0
product_id                           0
product_bought_count                 0
total_orders                         0
average_duration_btw_orders          0
total_products_ordered               0
num_distinct_products                0
avg_products_in_basket               0
orders                               0
reorders                             0
reorder_rate                         0
avg_pos_cart                         0
aisle                                0
department                           0
eval_set                             0
order_id                             0
reordered                      7645837
dtype: int64


missing values in test set
 user_id                        0
product_id                     0
product_bought_count           0
total_orders                   0
average_duration_btw_orders    0
total_products_ordered         0
num_distinct_products          0
avg_products_in_basket         0
o

**Missing value treatment**

There are missing values in target variable 'reordered' which indicates that these products were not reordered.
so filling these missing values with zero.

In [0]:
train['reordered'] = train['reordered'].fillna(value = 0)

In [0]:
# dropping useless column 'order_id'

train.drop(['order_id', 'eval_set', 'department', 'aisle'], axis = 1, inplace = True)
test.drop(['order_id', 'eval_set', 'department', 'aisle'], axis = 1, inplace = True)

#setting index

train = train.set_index(['user_id', 'product_id'])
test = test.set_index(['user_id', 'product_id'])

**Importing Tensorflow**

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
tf.set_random_seed(12)
from sklearn.model_selection import train_test_split

In [0]:
# seprating out the target variable

target = train['reordered']
train_df = train.drop('reordered', axis = 1)

In [0]:
#feature columns
features = train_df.columns

In [13]:
# spliting data in train and validation set

train_x, val_x, train_y, val_y  = train_test_split(train_df, target, test_size = 0.3, random_state = 12)

print('train_x shape is ', train_x.shape)
print('train_y shape is ', train_y.shape)
print('val_x shape is ', val_x.shape)
print('val_y shape is ', val_y.shape)

train_x shape is  (5932262, 10)
train_y shape is  (5932262,)
val_x shape is  (2542399, 10)
val_y shape is  (2542399,)


## **Tensorflow Gradient boosted tree model**

In [0]:
#preparing features
fc = tf.feature_column
feature_columns = []

for feature_name in features:
  feature_columns.append(fc.numeric_column(feature_name,
                                           dtype=tf.float32))


In [0]:

# creating input function
batch_size = 4096

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(batch_size)
       
    dataset = (dataset
      .repeat(n_epochs)
      .batch(batch_size)) 
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(train_x, train_y)
eval_input_fn = make_input_fn(val_x, val_y, shuffle=False, n_epochs=1)


In [16]:
params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  'center_bias': True
}

est = tf.estimator.BoostedTreesClassifier(feature_columns, **params)
est.train(train_input_fn, max_steps=100)
results = est.evaluate(eval_input_fn)
pd.Series(results).to_frame()


,0
accuracy,0.904732
accuracy_baseline,0.902548
auc,0.789215
auc_precision_recall,0.332072
average_loss,0.277762
label/mean,0.097452
loss,0.277766
precision,0.570979
prediction/mean,0.097812
recall,0.090131


Preparing test data for prediction

In [0]:
test_input_fn = lambda: tf.data.Dataset.from_tensors(dict(test))


Making prediction for test data

In [0]:
pred_test = list(est.predict(test_input_fn))
preds = np.array([pred['class_ids'] for pred in pred_test]).ravel()

**Preparing data for submission file**

In [19]:
sub = pd.read_csv('sample_submission.csv')
sub.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [0]:
del sub, train, train_x, train_y, val_x, val_y
test['prediction'] = preds

In [0]:
test = test.reset_index()
sub_df = test[['user_id', 'product_id', 'prediction']]
del test, preds

In [0]:
# getting test user_id and order_id

orders_df = pd.read_csv('orders.csv')
test_orders = orders_df.loc[orders_df.eval_set=='test',("user_id", "order_id") ]
del orders_df

In [23]:
sub_df = sub_df.merge(test_orders, on='user_id', how='left')
sub_df.head()

,user_id,product_id,prediction,order_id
0,3,248,0,2774568
1,3,1005,0,2774568
2,3,1819,0,2774568
3,3,7503,0,2774568
4,3,8021,0,2774568


In [0]:
# dropping user_id column and converting product id to integer

sub_df.drop('user_id', axis = 1, inplace = True)
sub_df.product_id = sub_df.product_id.astype(int)

In [0]:
# creating dictonary to store order_id and product_id of the products that each order will contain

dic = dict()
for row in sub_df.itertuples():
    if row.prediction== 1:
        try:
            dic[row.order_id] += ' ' + str(row.product_id)
        except:
            dic[row.order_id] = str(row.product_id)

for order in sub_df.order_id:
    if order not in dic:
        dic[order] = 'None'

**Creating submission**

In [28]:
#Convert the dictionary into a DataFrame
sub_df = pd.DataFrame.from_dict(dic, orient='index')

#Reset index
sub_df.reset_index(inplace=True)
#Set column names
sub_df.columns = ['order_id', 'products']

sub_df.head()

,order_id,products
0,1528013,21903
1,1376945,14947 27959
2,2161313,37710
3,1416320,5134 21903
4,1980631,6184 13575


In [0]:
sub_df.to_csv('instacart_submission.csv')